In [2]:
#importation des données dvf géolocalisées en csv

import os
import s3fs
import pandas as pd
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]

fs = s3fs.S3FileSystem(
    anon=True,
    client_kwargs={"endpoint_url": S3_ENDPOINT_URL}
)

with fs.open("renan/diffusion/dvf.csv", mode="rb") as f:
    df = pd.read_csv(f, dtype={'code_commune': "str"})

/tmp/ipykernel_39189/436545311.py:14: DtypeWarning: Columns (12,14,16,17,18,20,22,24,26,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, dtype={'code_commune': "str"})


In [3]:
#On se limite aux maisons et appartements

df = df[df['type_local'].isin(['Maison', 'Appartement'])]
print(df)
df=df.reset_index()

          id_mutation date_mutation  numero_disposition nature_mutation  \
17             2020-6    2020-07-02                   1           Vente   
31             2020-8    2020-07-01                   1           Vente   
32             2020-9    2020-07-02                   1           Vente   
36            2020-11    2020-07-03                   1           Vente   
37            2020-12    2020-07-02                   1           Vente   
...               ...           ...                 ...             ...   
20102723  2025-502875    2025-06-23                   1           Vente   
20102725  2025-502876    2025-06-25                   1           Vente   
20102726  2025-502877    2025-06-25                   1           Vente   
20102734  2025-502880    2025-06-27                   1           Vente   
20102735  2025-502881    2025-06-27                   1           Vente   

          valeur_fonciere  adresse_numero adresse_suffixe  \
17                77000.0           34

In [4]:
comptage_df = df['id_mutation'].value_counts().reset_index()
comptage_df.columns = ['id_mutation', 'nombre_de_lignes']
print(comptage_df)

id_uniques = comptage_df[comptage_df['nombre_de_lignes'] == 1]
print(id_uniques.shape)

print(id_uniques)
df_sans_lots=df[df['id_mutation'].isin(id_uniques['id_mutation'])]
print(df_sans_lots)

         id_mutation  nombre_de_lignes
0        2021-687176              9290
1        2022-919567              7657
2        2022-909252              7477
3        2021-342324              7112
4        2020-491811              6173
...              ...               ...
4861632    2022-7412                 1
4861633    2022-7410                 1
4861634    2022-7409                 1
4861635    2022-7405                 1
4861636    2022-7419                 1

[4861637 rows x 2 columns]
(4133749, 2)
         id_mutation  nombre_de_lignes
727888   2023-581082                 1
727889   2023-510332                 1
727890   2023-580244                 1
727891   2023-510334                 1
727892   2023-579686                 1
...              ...               ...
4861632    2022-7412                 1
4861633    2022-7410                 1
4861634    2022-7409                 1
4861635    2022-7405                 1
4861636    2022-7419                 1

[4133749 rows x 2 colu

In [4]:
#les codes communes n'ont pas tous le même type ; certains sont des ints, d'autres des strings. On convertit l'ensemble en string pour plus de cohérence(on 
#ne peut pas faire l'opération inverse ; les codes corses commencent par 2A ou 2B !)

def inttostr(valeur):
    if isinstance(valeur, float):
        return str(valeur)
    if isinstance(valeur, int):
        return str(valeur)
    else:
        # Si c'est déjà une str, None, float, etc., on le retourne tel quel
        return valeur
print(inttostr('433'))

df.loc[:,'code_commune'] = df['code_commune'].apply(inttostr)
print(df['code_commune'])

433
0           1130
1           1451
2           1364
3           1053
4           1177
           ...  
6247210    75111
6247211    75112
6247212    75116
6247213    75113
6247214    75105
Name: code_commune, Length: 6247215, dtype: object


In [6]:
#Calcul du nombre de ventes commune

ventes_par_commune = df['code_commune'].value_counts().reset_index(name='nombre')

#Résultats : on remarque on particulier que Toulouse est la commune comptant le plus de ventes entre 2020 et 2025(50684) !!!

print(ventes_par_commune)

print( ventes_par_commune["nombre"][ventes_par_commune["code_commune"]=='2A004'])

print(ventes_par_commune['nombre'].max())

print(ventes_par_commune['nombre'].idxmax())

print(ventes_par_commune['code_commune'][ventes_par_commune['nombre'].idxmax()])

print(ventes_par_commune['nombre'][ventes_par_commune['nombre'].idxmax()])


      code_commune  nombre
0            31555   50684
1            06088   46697
2            44109   30331
3            33063   30077
4            34172   28927
...            ...     ...
33265        52549       1
33266        21136       1
33267        21653       1
33268        52053       1
33269        52505       1

[33270 rows x 2 columns]
145    4880
Name: nombre, dtype: int64
50684
0
31555
50684


In [42]:
#On récupère désormais les données de population ; cependant certaines valeurs ne sont pas renseignés. On remarque aussi que certaines communes ont des populations trop faibles
#pour que le ratio considéré ventes entre 2020 et 2050/population en 2019 ait du sens ; en particulier, la commune de Bezonvaux a zéro habitants !!! 
# On impose donc un nombre d'habitants minimum arbitraire, et on exclut les communes de populations trop faibles ou non renseignés

df_pop=pd.read_excel("popcommunes.xlsx")
df_pop.rename(columns={'codgeo': 'code_commune'}, inplace=True)
print(df_pop)

cols = ["p19_pop"]
df_pop[cols].isna().sum()
lignes_na = df_pop[df_pop[cols].isna().any(axis=1)]
print(lignes_na)
print(lignes_na.shape)

       objectid  reg dep    cv code_commune                 libgeo  p13_pop  \
0        115658   52  85  8502        85062            Châteauneuf    968.0   
1        115659   26  58  5808        58300                   Urzy   1839.0   
2        115660   43  70  7012        70137  Chassey-lès-Montbozon    218.0   
3        115661   21  51  5123        51649      Vitry-le-François  13174.0   
4        115662   11  78  7811        78638         Vaux-sur-Seine   4749.0   
...         ...  ...  ..   ...          ...                    ...      ...   
34990    110251   31  62  6225        62327         Febvin-Palfart    574.0   
34991    110252   73  12  1201        12156            Montpeyroux    550.0   
34992    110253   54  86  8610        86138                Luchapt    275.0   
34993    110254   72  33  3320        33184                Générac    579.0   
34994    110255   72  24  2409        24354       La Roche-Chalais   2932.0   

       p14_pop  p15_pop  p16_pop  p17_pop  p18_pop 

In [49]:
#Problème avec le fichier de données de population ; certains codes communes sont à six chiffres ! en effet dans les DOMS le numéro commence par un code département à trois chiffres plutôt qu'à deux(pour différencier les DOMs) !
#cela n'est pas un problème en soit mais la convention adoptée dans le fichier dvf est différente, on ne rajoute pas le troisième chiffre

#lignes à six chiffres

print(df_pop[df_pop['code_commune']=='974421'])
lignes_6_chiffres = df_pop[df_pop["code_commune"].str.len() == 6]
print(lignes_6_chiffres)

#code commune différent dans df_pop et df pour la commune de Salazie, à la Réunion ; 974421 dans df_pop, 97421 ; la Réunion est perçue comme le département
#974 dans df_pop, on va supprimer le 4

print(df_pop[df_pop['code_commune']=='974421']['libgeo'])
print(df[df['code_commune']=='97421']['nom_commune'])

      objectid  reg  dep     cv code_commune   libgeo  p13_pop  p14_pop  \
3786    131030    4  974  97406       974421  Salazie   7226.0   7132.0   

      p15_pop  p16_pop  p17_pop  p18_pop  p19_pop  p20_pop  p21_pop  
3786   7384.0   7400.0     7312   7224.0   7136.0   7310.0   7243.0  
       objectid  reg  dep     cv code_commune                  libgeo  \
84       126394    3  973    973       973353             Maripasoula   
192      126587    3  973    973       973313  Montsinéry-Tonnegrande   
206      126601    3  973    973       973358              Saint-Élie   
210      126605    1  971  97114       971102           Anse-Bertrand   
748      130678    1  971  97195       971106              Bouillante   
...         ...  ...  ...    ...          ...                     ...   
33141    115576    2  972    972       972225            Saint-Pierre   
33257    111336    4  974  97401       974404           L' Étang-Salé   
33557    111383    2  972    972       972201      L

In [51]:
#Pour remédier au problème précédent, il suffit de supprimer le troisième chiffre de chacun des codes communes à six chiffres.

def enleverchiffreDOMs(code):
    if len(code) == 6:
        # retirer le 3ème caractère (index 2)
        return code[:2] + code[3:]
    return code

# appliquer sur la colonne
df_pop["code_commune"] = df_pop["code_commune"].apply(enleverchiffreDOMs)

print(df_pop)
lignes_6_chiffres = df_pop[df_pop["code_commune"].str.len() == 6]
print(lignes_6_chiffres)
print(df_pop[df_pop['code_commune']=='974421'])

       objectid  reg dep    cv code_commune                 libgeo  p13_pop  \
0        115658   52  85  8502        85062            Châteauneuf    968.0   
1        115659   26  58  5808        58300                   Urzy   1839.0   
2        115660   43  70  7012        70137  Chassey-lès-Montbozon    218.0   
3        115661   21  51  5123        51649      Vitry-le-François  13174.0   
4        115662   11  78  7811        78638         Vaux-sur-Seine   4749.0   
...         ...  ...  ..   ...          ...                    ...      ...   
34990    110251   31  62  6225        62327         Febvin-Palfart    574.0   
34991    110252   73  12  1201        12156            Montpeyroux    550.0   
34992    110253   54  86  8610        86138                Luchapt    275.0   
34993    110254   72  33  3320        33184                Générac    579.0   
34994    110255   72  24  2409        24354       La Roche-Chalais   2932.0   

       p14_pop  p15_pop  p16_pop  p17_pop  p18_pop 

In [52]:
#On joint les données en se restreignant aux communes dont on connaît les populations et de populations pas trop faibles. Le min est arbitraire est peut être changé

ventes_par_commune_par_habitant=pd.merge(ventes_par_commune, df_pop, on='code_commune')
ventes_par_commune_par_habitant = ventes_par_commune_par_habitant[
    (ventes_par_commune_par_habitant['p19_pop'].notna()) &
    (ventes_par_commune_par_habitant['p19_pop'] > 1000)
]
ventes_par_commune_par_habitant['ventes par habitants par commune']=ventes_par_commune_par_habitant['nombre']/ventes_par_commune_par_habitant['p19_pop']

print(ventes_par_commune_par_habitant['ventes par habitants par commune'])

print(ventes_par_commune_par_habitant['ventes par habitants par commune'].max())

print(ventes_par_commune_par_habitant['ventes par habitants par commune'].idxmax())

print(ventes_par_commune_par_habitant['code_commune'][ventes_par_commune_par_habitant['ventes par habitants par commune'].idxmax()])

0        0.102710
1        0.136274
2        0.095139
3        0.115256
4        0.097878
           ...   
22363    0.014115
26593    0.015238
30690    0.005638
31746    0.000683
32063    0.000527
Name: ventes par habitants par commune, Length: 9422, dtype: float64
1.172281776416539
611
38191


In [53]:
#tentative de récupération des prix au m2
#Remarque : on ne considère que le ratio valeur foncière/surface batîe de la maison ou de l'appartement, puisqu'on ignore les dépendances ou autres terrains additionnels.
#on sur-estime donc la valeur au m2. On ignore les lots(on utilise df_sans_lots qui ne contient pas les observations dont l'id_mutation apparait plusieurs fois)
#à la fois parce qu'on ne peut pas retrouver la valeur foncière des éléments individuels, et pour se concentrer sur les logements anciens
df_sans_lots = df_sans_lots.copy()
print(df_sans_lots.shape)

df_sans_lots = df_sans_lots[
    (df_sans_lots['surface_reelle_bati'].notna()) &
    (df_sans_lots['valeur_fonciere'].notna()) &
    (df_sans_lots['surface_reelle_bati'] > 10)
]



df_sans_lots['rapport valeur foncière et surface bâtie']=df_sans_lots['valeur_fonciere']/df_sans_lots['surface_reelle_bati']

print(df_sans_lots['rapport valeur foncière et surface bâtie'])

print(df_sans_lots['rapport valeur foncière et surface bâtie'].max())

print(df_sans_lots['rapport valeur foncière et surface bâtie'].idxmax())

print(df_sans_lots.loc[df_sans_lots['rapport valeur foncière et surface bâtie'].idxmax()])

#problème ; valeurs monstrueuses de certains logements, visiblement des immeubles entiers plutôt que des logements(ci-dessus,immeuble en construction de presque 1 milliard...)


moyenne_par_commune=df_sans_lots.groupby('code_commune')['rapport valeur foncière et surface bâtie'].mean().reset_index()

print(moyenne_par_commune['rapport valeur foncière et surface bâtie'].max())

print(moyenne_par_commune['code_commune'].loc[moyenne_par_commune['rapport valeur foncière et surface bâtie'].idxmax()])



# Calcul de la médiane par commune
mediane_par_commune = df_sans_lots.groupby('code_commune')['rapport valeur foncière et surface bâtie'].median().reset_index()

# Valeur maximale de la médiane
print(mediane_par_commune['rapport valeur foncière et surface bâtie'].max())

# Code de la commune avec la médiane maximale
print(mediane_par_commune['code_commune'][mediane_par_commune['rapport valeur foncière et surface bâtie'].idxmax()])

(4125604, 42)
0            669.565217
1           2286.885246
2           2220.625000
3           1966.406977
4           1666.666667
               ...     
6247210    11928.571429
6247211    11363.636364
6247212    14891.304348
6247213     9016.393443
6247214    11710.342553
Name: rapport valeur foncière et surface bâtie, Length: 4125604, dtype: float64
15937500.0
5816283
index                                                      18656556
id_mutation                                            2024-1198671
date_mutation                                            2024-06-27
numero_disposition                                                1
nature_mutation                                               Vente
valeur_fonciere                                         255000000.0
adresse_numero                                                 42.0
adresse_suffixe                                                 NaN
adresse_nom_voie                                       AV MONTAIGNE
adresse_cod

In [54]:
#meilleure alternative au troncage précédent ; on tronque par commune, on touve alors des médianes et moyennes très proches, maximale pour le sixième arrondissement parisien, c'est cohérent !

# Calculer les quantiles 0.025 et 0.975 par commune
quantiles_par_commune = (
    df_sans_lots
    .groupby('code_commune')['rapport valeur foncière et surface bâtie']
    .quantile([0.025, 0.975])
    .unstack()
    .reset_index()
    .rename(columns={0.025: 'quantile_025', 0.975: 'quantile_975'})
)

print(quantiles_par_commune)

# Fusionner les quantiles avec df_sans_lots
df_sans_lots_tronqué = df_sans_lots.merge(
    quantiles_par_commune,
    on='code_commune',
    how='left'
)

df_sans_lots_tronqué = df_sans_lots_tronqué[
    (df_sans_lots_tronqué['rapport valeur foncière et surface bâtie'] >= df_sans_lots_tronqué['quantile_025']) &
    (df_sans_lots_tronqué['rapport valeur foncière et surface bâtie'] <= df_sans_lots_tronqué['quantile_975'])
]

# Calcul de la moyenne par commune (tronqué)
moyenne_par_commune_tronqué = df_sans_lots_tronqué.groupby('code_commune')['rapport valeur foncière et surface bâtie'].mean().reset_index()

# Affichage de la moyenne maximale et de la commune associée
print(moyenne_par_commune_tronqué['rapport valeur foncière et surface bâtie'].max())
print(moyenne_par_commune_tronqué['code_commune'].loc[moyenne_par_commune_tronqué['rapport valeur foncière et surface bâtie'].idxmax()])

# Calcul de la médiane par commune (tronqué)
mediane_par_commune_tronqué = df_sans_lots_tronqué.groupby('code_commune')['rapport valeur foncière et surface bâtie'].median().reset_index()

# Affichage de la médiane maximale et de la commune associée
print(mediane_par_commune_tronqué['rapport valeur foncière et surface bâtie'].max())
print(mediane_par_commune_tronqué['code_commune'][mediane_par_commune_tronqué['rapport valeur foncière et surface bâtie'].idxmax()])

      code_commune  quantile_025  quantile_975
0            01001   1402.258065   4650.588503
1            01002    394.560643   3523.392756
2            01004   1017.798713   3943.055556
3            01005   1107.758621   4353.815194
4            01006    272.916667   2423.611111
...            ...           ...           ...
33081        97420    434.588798   4471.316033
33082        97421    191.712454   3174.025974
33083        97422    673.073579   5184.047619
33084        97423    177.530190   7281.037415
33085        97424    230.390338   5416.666667

[33086 rows x 3 columns]
14889.448668014387
75106
14752.04918032787
75106


In [ ]:
import pandas

#On crée un dataframe complet contenant toutes les données pertinentes par commune 
#Attention : les données sur les ventes par habitant ont été crées plus tôt en se restreignant aux communes avec au moins un certain nombre d'ventes_par_commune_par_habitant_simple
#Remonter et changer de seuil si on veut élargir au restreindre le champ des données
#!!!Dans tous les cas il y aura pas mal de valeurs manquantes pour les données en question, bien faire gaffe à ce qu'on étudie!!!



pop_par_commune=df_pop[["code_commune", "p19_pop","libgeo"]]
ventes_par_commune_par_habitant_simple=ventes_par_commune_par_habitant[["code_commune", "ventes par habitants par commune"]]

dfs=[pop_par_commune, moyenne_par_commune_tronqué, mediane_par_commune_tronqué, ventes_par_commune, ventes_par_commune_par_habitant_simple]
print(ventes_par_commune)
print(ventes_par_commune_par_habitant_simple)

df_final=pandas.merge(pop_par_commune, ventes_par_commune, how='outer', on="code_commune")
df_final.rename(columns={'nombre': 'nombre de ventes dans la commune'}, inplace=True)
df_final.rename(columns={'libgeo': 'nom de la commune'}, inplace=True)
df_final.rename(columns={'p19_pop': 'population en 2019'}, inplace=True)
print(df_final)
df_final=pandas.merge(df_final, ventes_par_commune_par_habitant_simple, how='outer', on="code_commune")
df_final.rename(columns={'ventes par habitants par commune': 'nombre de ventes par habitant en 2019 dans la commune'}, inplace=True)
df_final=pandas.merge(df_final, mediane_par_commune_tronqué , how='outer', on="code_commune")
df_final.rename(columns={'rapport valeur foncière et surface bâtie': 'médiane tronquée du prix au m2'}, inplace=True)
df_final=pandas.merge(df_final, moyenne_par_commune_tronqué , how='outer', on="code_commune")
df_final.rename(columns={'rapport valeur foncière et surface bâtie': 'moyenne tronquée du prix au m2'}, inplace=True)
print(df_final)


      code_commune  nombre
0            31555   50684
1            06088   46697
2            44109   30331
3            33063   30077
4            34172   28927
...            ...     ...
33265        52549       1
33266        21136       1
33267        21653       1
33268        52053       1
33269        52505       1

[33270 rows x 2 columns]
      code_commune  ventes par habitants par commune
0            31555                          0.102710
1            06088                          0.136274
2            44109                          0.095139
3            33063                          0.115256
4            34172                          0.097878
...            ...                               ...
22363        97133                          0.014115
26593        97215                          0.015238
30690        97110                          0.005638
31746        97357                          0.000683
32063        97360                          0.000527

[9422 rows x 